### Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:

- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [2]:
films = pd.read_csv('ratings2.csv')
films.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
def class_films(df):
    
    if df.rating <= 2:
        return 'low'
    
    elif 2 < df.rating <= 4:
        return 'medium'
    
    elif 4 < df.rating:
        return 'high'

In [7]:
films['class'] = films.apply(class_films, axis=1)
films.head()

,userId,movieId,rating,timestamp,class
0,1,1,4.0,964982703,medium
1,1,3,4.0,964981247,medium
2,1,6,4.0,964982224,medium
3,1,47,5.0,964983815,high
4,1,50,5.0,964982931,high


In [8]:
films.tail()

,userId,movieId,rating,timestamp,class
100831,610,166534,4.0,1493848402,medium
100832,610,168248,5.0,1493850091,high
100833,610,168250,5.0,1494273047,high
100834,610,168252,5.0,1493846352,high
100835,610,170875,3.0,1493846415,medium


### Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [33]:
key_df = pd.read_csv('keywords.csv')
key_df.head(10)

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195
5,одноклассники моя страница,19976138
6,майл,15144673
7,авито,14786593
8,переводчик,14453469
9,яндекс,13344734


In [76]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'] }

In [77]:
def geo_logs(df):
    
    for i in geo_data['Центр']:        
        if i in df['keyword']:
            return 'Центр'
    
    for i in geo_data['Северо-Запад']:
        if i in df['keyword']:
            return 'Северо-Запад'
           
    for i in geo_data['Дальний Восток']:
        if i in df['keyword']:
            return 'Дальний Восток'
        
    return 'undefined'

In [78]:
key_df['region'] = key_df.apply(geo_logs, axis=1)
key_df.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [79]:
key_df['region'].value_counts().head()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

In [80]:
key_df.groupby('region').count().head()

,keyword,shows
region,,
undefined,99253,99253
Дальний Восток,94,94
Северо-Запад,266,266
Центр,387,387


In [81]:
key_df_filtered = key_df[ (key_df['region']=='Центр') | (key_df['region']=='Северо-Запад') | \
                     (key_df['region']=='Дальний Восток') ]
key_df_filtered.head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток


In [84]:
key_df_filtered = key_df[ key_df['region'].isin(['Центр', 'Северо-Запад','Дальний Восток']) ]
key_df_filtered.head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток


### Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

    1. для каждой строки пройдите по всем годам списка years
    2. если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    3. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
    
1. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

2. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [85]:
films2 = pd.read_csv('ratings2.csv')
mov = pd.read_csv('movies2.csv')
mov.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [89]:
films2.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [100]:
mov_merged = mov.merge(films2, on='movieId', how='left')
mov_merged.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [161]:
years = []
for i in range(1950, 2011):
    years.append(i)

In [166]:
def production_year(df):
    
    for i in years: 
        
        if str(i) in df['title']:
            
            return i
        
    return '1900'

In [167]:
mov_merged['years'] = mov_merged.apply(production_year, axis=1)
mov_merged.head()

,movieId,title,genres,userId,rating,timestamp,years
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08,1995
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08,1995
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09,1995
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09,1995
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09,1995


In [185]:
mov_merged.groupby('years').mean()[['rating']].sort_values(by='rating', ascending=False).head(15)

,rating
years,
1957,4.039535
1954,4.009191
1962,3.969466
1952,3.953125
1972,3.944293
1964,3.940160
1974,3.935622
1967,3.922572
1975,3.879121


__Комментарий__: Ниже я пробовал записывал столбец year не указанным в задании способом, а забирать его из title с помощью регулярных выражений, но что-то пошло не так — не понимаю, почему возникает ошибка с 'NoneType' object, хотя по моим представлениям там должна возвращаться строка 

In [126]:
import re

In [147]:
def get_year(df):
    
    y = re.search(r'(?<=\()\d{4}', df['title'])
    
    return y.group()

In [148]:
mov_merged['years'] = mov_merged.apply(get_year, axis=1)
mov_merged.head()

AttributeError: 'NoneType' object has no attribute 'group'

In [162]:
test = 	'Toy Story (1995)'
x = re.search(r'(?<=\()\d{4}', test)
type(x.group())

str